In [7]:
import requests
from bs4 import BeautifulSoup
import pgeocode
import pandas as pd 

In [138]:
zip_code = input("Zip Code: ")

Zip Code:  22015


In [139]:
nomi = pgeocode.Nominatim("US")
lat=nomi.query_postal_code(zip_code)["latitude"]
long=nomi.query_postal_code(zip_code)["longitude"]

In [140]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat={}&lon={}".format(str(lat),str(long)))
soup = BeautifulSoup(page.content, "html.parser")
seven_day = soup.find(id = "seven-day-forecast")
forecast_items = seven_day.find_all(class_ = "tombstone-container")


In [141]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Monday',
 'MondayNight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight']

In [142]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

In [143]:
weather = pd.DataFrame({
    "period": periods, 
    "short_descs": short_descs,
    "temp": temps,
    "desc": descs
})
weather

,period,short_descs,temp,desc
0,Tonight,Mostly Clear,Low: 32 °F,"Tonight: Mostly clear, with a low around 32. N..."
1,Monday,IncreasingClouds,High: 48 °F,"Monday: Increasing clouds, with a high near 48..."
2,MondayNight,Partly Cloudy,Low: 29 °F,"Monday Night: Partly cloudy, with a low around..."
3,Tuesday,Slight ChanceRain thenRain,High: 46 °F,"Tuesday: Rain, mainly after 10am. High near 4..."
4,TuesdayNight,Rain,Low: 41 °F,"Tuesday Night: Rain, mainly before 4am. Low a..."
5,Wednesday,Partly Sunny,High: 51 °F,"Wednesday: Partly sunny, with a high near 51."
6,WednesdayNight,Partly Cloudy,Low: 33 °F,"Wednesday Night: Partly cloudy, with a low aro..."
7,Thursday,Sunny,High: 45 °F,"Thursday: Sunny, with a high near 45."
8,ThursdayNight,Mostly Clear,Low: 28 °F,"Thursday Night: Mostly clear, with a low aroun..."


In [144]:
current_conditions = soup.find(id = "current_conditions_detail")
items = [i.get_text() for i in current_conditions.select("td")]
items ={items[i]: items[i + 1] for i in range(0, len(items), 2)}
items

{'Humidity': '47%',
 'Wind Speed': 'NW 10 mph',
 'Barometer': '30.21 in (1023.3 mb)',
 'Dewpoint': '23°F (-5°C)',
 'Visibility': '10.00 mi',
 'Wind Chill': '36°F (2°C)',
 'Last update': '\n                13 Nov 7:55 pm EST            '}

In [145]:
conditions = pd.Series(items).str.strip("\n").str.strip()
conditions

Humidity                        47%
Wind Speed                NW 10 mph
Barometer      30.21 in (1023.3 mb)
Dewpoint                23°F (-5°C)
Visibility                 10.00 mi
Wind Chill               36°F (2°C)
Last update      13 Nov 7:55 pm EST
dtype: object

In [130]:
import pymongo

In [136]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
weatherDB = myclient["Weather"]
conditions = myclient["Conditions"]